In [1]:
library(dplyr)
library(MuSiC) %>% suppressMessages()
library(fastSave) %>% suppressMessages()
library(pbmcapply) %>% suppressMessages()
library(Seurat) %>% suppressMessages()
library(SingleCellExperiment) %>% suppressMessages()


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
# load("~/代表性方法测试/BayesPrism/tutorial/tutorial.gbm.rdata")
combine.dgedata_150=readRDS.lbzip2('~/ReCIDE/benchmark测试/high_res_CRC/ref_data/ref_all.rdsFS',n.cores = 200)
patient_names=unique(combine.dgedata_150@meta.data['PatientTypeID'])


In [7]:

func_mu<-function(i){
  print(paste('第',i,'个'))
  bulk_seurat<-subset(combine.dgedata_150,PatientTypeID == patient_names[i,1])
    
  sc_seurat<-subset(combine.dgedata_150,PatientTypeID != patient_names[i,1])

  bulk.mtx<-as.matrix(apply(as.data.frame(bulk_seurat@assays[["RNA"]]@counts),1,sum))
  bulk.mtx<-cbind(bulk.mtx,bulk.mtx)
  
  
  #单细胞参考集是sce的形式
  sce <- SingleCellExperiment(as.matrix(sc_seurat@assays$RNA@counts),
                              colData=DataFrame(label=sc_seurat@meta.data),
                              rowData=DataFrame(length=row.names(sc_seurat@assays$RNA@counts)))
  names(assays(sce))<-'counts'
  
  #在music前要先确定bulk和sc是否只有一列，只有一列的话无法运行（包括sample和cluster数是否为1）
  music_results_in = music_prop(bulk.mtx = bulk.mtx, sc.sce = sce, clusters = 'label.usetype',
                                samples = 'label.PatientTypeID',verbose = F)
    return(music_results_in)
}
music_results=pbmclapply(1:nrow(patient_names),func_mu,mc.cores=4)
names(music_results)<-patient_names[,1]

In [ ]:
saveRDS(music_results,file='~/ReCIDE/benchmark测试/high_res_CRC/music/music_tutorial/music_output_parallel.rds')

In [19]:
# music_results_parallel=music_results

In [9]:
music_results=readRDS('~/ReCIDE/benchmark测试/high_res_CRC/music/music_tutorial/music_output_parallel.rds')

In [10]:
###结果整合
for (i in 1:length(music_results)) {
  music_results[[i]]<-music_results[[i]][['Est.prop.weighted']][1,]
}
prd<-music_results

df_merge<-as.data.frame(prd[[1]])
colnames(df_merge)[1]<-names(prd)[1]

for(j in 2:length(prd)){
  
  prd[[j]]<-as.data.frame(prd[[j]])
  df_merge<-merge(df_merge, prd[[j]], by = "row.names", all = TRUE)
  row.names(df_merge)<-df_merge[,1]
  df_merge<-df_merge[,-1]
  colnames(df_merge)[j]<-names(prd)[j]
}



df_merge<-as.data.frame(df_merge)
df_merge<-df_merge[,sort(colnames(df_merge))]
df_merge<-df_merge[sort(row.names(df_merge)),]
prd_df=df_merge

In [12]:
key_df<- readRDS("~/ReCIDE/benchmark测试/high_res_CRC/EXP_and_KEY/KEY.rds")
key_df=key_df[sort(row.names(key_df)),sort(colnames(key_df))]


RMSE_vec=c()
for(i in 1:length(key_df)){
  RMSE_vec[i]=ModelMetrics::rmse(key_df[,i],prd_df[,i])
  # RMSE_vec[i]=cor(key_df[,i],prd_df[,i])
  
}
mean(RMSE_vec)

[1] 0.03823223

In [11]:
saveRDS(prd_df,file='/home/lmh/ReCIDE/benchmark测试/high_res_CRC/music/music_tutorial/prd_music_df.rds')